# S.M.A.R.T QUESTIONS

1. Analisis Kunjungan Pasien
- Apakah ada pola tertentu dalam frekuensi kunjungan pasien berdasarkan tanggal pemeriksaan (tanggal_periksa)?
- Berapa rata-rata jumlah kunjungan per pasien dalam satu bulan? Apakah ada pasien yang sering berulang kali datang?
2. Analisis Berdasarkan Klinik dan Dokter
- Klinik atau poli mana yang paling sering dikunjungi pasien? Adakah hubungan antara jenis kunjungan (jenis_kunjungan) dengan klinik tertentu?
- Dokter mana yang memiliki jumlah pasien terbanyak? Adakah hubungan antara jumlah kunjungan dengan hari tertentu?
3. Analisis Status BPJS dan Keberhasilan Pengiriman
- Berapa persentase keberhasilan dan kegagalan status pengiriman data (status_kirim)? Apakah ada tren tertentu dalam data yang gagal terkirim?
- Adakah pola tertentu dalam kegagalan pengiriman berdasarkan jenis kunjungan atau poli tertentu?
4. Analisis Berdasarkan Jenis Kunjungan dan BPJS
- Jenis kunjungan (jenis_kunjungan) mana yang paling dominan dalam dataset? Apakah pasien dengan kunjungan tertentu lebih sering mengalami kegagalan pengiriman?
- Apakah pasien dengan nomor kartu BPJS yang sama sering berkunjung ke berbagai klinik atau hanya ke klinik tertentu?
5. Analisis Perbandingan Kinerja Berdasarkan Tanggal
- Apakah ada peningkatan atau penurunan jumlah pasien pada tanggal tertentu (misalnya, akhir pekan vs hari kerja)?
- Adakah hari tertentu dengan volume pasien tinggi yang menyebabkan peningkatan kegagalan pengiriman?
6. Identifikasi Pasien dengan Kunjungan Berulang dan Nomor BPJS
- Berapa banyak pasien yang melakukan lebih dari satu kunjungan dalam periode yang sama? Apakah mereka lebih rentan terhadap kegagalan pengiriman status?
- Apakah pasien dengan nomor BPJS tertentu lebih sering mengalami masalah status kirim?

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import matplotlib.patches as mpatches 
from matplotlib.colors import LinearSegmentedColormap
import warnings 
import missingno as mo


warnings.filterwarnings('ignore')
sns.set(style='whitegrid')
plt.style.use('Solarize_Light2')


In [4]:
df = pd.read_csv('./data/juli 2024.csv')

In [5]:
pd.DataFrame(df)

,no_rawat,tanggal_periksa,nm_poli,nm_dokter,nm_pasien,no_rkm_medis,png_jawab,nomor_kartu,nomor_referensi,kodebooking,jenis_kunjungan,status_kirim
0,2024/07/01/000004,31/07/2024,KLINIK MATA,"dr. H. Rezekian, Sp.M",MARNI Hj. Ny,97522.0,BPJS Kesehatan,1080583615,170603010624Y000093,1706R00620240731000020MAT004,3.0,Sudah
1,2024/07/01/000004,15/07/2024,KLINIK MATA,"dr. H. Rezekian, Sp.M",MARNI Hj. Ny,97522.0,BPJS Kesehatan,1080583615,170603010724Y000496,1706R00620240715093247JIW005MJKN,1.0,Gagal
2,2024/07/01/000004,19/07/2024,KLINIK MATA,"dr. H. Rezekian, Sp.M",MARNI Hj. Ny,97522.0,BPJS Kesehatan,1080583615,170603010624Y000093,1706R00620240719000016MAT001,3.0,Sudah
3,2024/07/01/000004,12/7/2024,KLINIK MATA,"dr. H. Rezekian, Sp.M",MARNI Hj. Ny,97522.0,BPJS Kesehatan,1080583615,1706R0060624K003795,1706R00620240708034048KLT001MJKN,3.0,Gagal
4,2024/07/01/000004,15/07/2024,KLINIK MATA,"dr. H. Rezekian, Sp.M",MARNI Hj. Ny,97522.0,BPJS Kesehatan,1080583615,170603010724Y000496,1706R00620240715000190JIW005,3.0,Sudah
...,...,...,...,...,...,...,...,...,...,...,...,...
1354,2024/07/04/000121,15/07/2024,KLINIK PENYAKIT DALAM,"dr. CHARISMA BIMARA COZY, M.Kes, Sp.PD","RIDUAN, Tn",224148.0,BPJS Kesehatan,2919164264,170604020624P001102,1706R00620240715000074INT003,3.0,Sudah
1355,2024/07/04/000121,22/07/2024,KLINIK PENYAKIT DALAM,"dr. CHARISMA BIMARA COZY, M.Kes, Sp.PD","RIDUAN, Tn",224148.0,BPJS Kesehatan,2919164264,170604020624P001102,1706R00620240722000120INT014,3.0,Sudah
1356,2024/07/04/000121,4/7/2024,KLINIK PENYAKIT DALAM,"dr. CHARISMA BIMARA COZY, M.Kes, Sp.PD","RIDUAN, Tn",224148.0,BPJS Kesehatan,2919164264,170604020624P001102,1706R00620240704000121INT018,3.0,Sudah
1357,2024/07/04/000123,25/07/2024,KLINIK MATA,"dr. H. Rezekian, Sp.M",H.JONI KARLI,194620.0,BPJS Kesehatan,3304753233,170607010424P001248,1706R00620240725000061MAT019,3.0,Sudah
